In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

In [2]:
df=pd.read_csv('C:\\Users\\My1\\Desktop\\latestst2\\Spamtest1.csv')
df.head()

,type,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df=df.replace(['ham','spam'],[0,1])
df.head()

,type,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df['Count']=0
for i in np.arange(0,len(df.text)):
    df.loc[i,'Count'] = len(df.loc[i,'text'])
df.head()

,type,text,Count
0,0,"Go until jurong point, crazy.. Available only ...",111
1,0,Ok lar... Joking wif u oni...,29
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,0,U dun say so early hor... U c already then say...,49
4,0,"Nah I don't think he goes to usf, he lives aro...",61


In [5]:
corpus = []
ps = PorterStemmer()
df['type'].value_counts()

0    4825
1     747
Name: type, dtype: int64

In [8]:
for i in range (0,5572):
    m = df['text'][i]
    m = re.sub('\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailadd', df['text'][i])
    m = re.sub('(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'urladd', df['text'][i])
    m = re.sub('£|\$', 'moneysymbols', df['text'][i])
    m = re.sub('\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', 'phoneno', df['text'][i])
    m = re.sub('\d+(\.\d+)?', 'numbers', df['text'][i])
    m = re.sub('[^\w\d\s]', ' ', df['text'][i])    
    m = m.lower().split()    
    m = [ps.stem(word) for word in m if not word in set(stopwords.words('english'))]  
    m = ' '.join(m)   
    corpus.append(m)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
cv = CountVectorizer()
x = cv.fit_transform(corpus).toarray()

In [11]:
y = df['type']
print (y.value_counts())

print(y[0])
print(y[1])

0    4825
1     747
Name: type, dtype: int64
0
0


In [12]:
le = LabelEncoder()
y = le.fit_transform(y)

print(y[0])
print(y[1])

0
0


In [13]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y,test_size= 0.15, random_state = 0)

In [14]:
svm=SVC(kernel='linear',gamma='auto')

In [15]:
svm.fit(xtrain,ytrain)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [17]:
yp=svm.predict(xtest)
cm = confusion_matrix(ytest, yp)

print(cm)
print ("Accuracy : %0.5f \n\n" % accuracy_score(ytest, yp))

[[722   0]
 [  9 105]]
Accuracy : 0.98923 




In [19]:
f1k=["Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005.Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"]
final=cv.transform(f1k).toarray()
if(svm.predict(final)==0):
    print("Not Spam")
else:
    print("Spam")

Spam
